In [33]:
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], ".."))
from modt.modt import MoDT
from modt._initialization import *
from modt.visualization import *
from modt.utility import *

import pickle
from timeit import default_timer as timer

import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import normalize
from sklearn.model_selection import RepeatedKFold

In [34]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth', None)

In [35]:
df = pd.read_pickle("dataframes/df_intersections_gate_reduction.pd")

In [36]:
df

,dataset,DT-LDA,DT-LDAm,DT-LR,DT-LRm,DT-XGB,DT-L,LDA-LDAm,LDA-LR,LDA-LRm,...,LDAm-LR,LDAm-LRm,LDAm-XGB,LDAm-L,LR-LRm,LR-XGB,LR-L,LRm-XGB,LRm-L,XGB-L
0,breast_cancer_input.np,0.0,0.0,1.0,1.0,0.5,0.5,1.0,0.0,0.0,...,0.0,0.0,0.5,0.0,1.0,0.5,0.5,0.5,0.5,0.5
1,iris_input.pd,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,...,1.0,1.0,1.0,0.5,1.0,1.0,0.5,1.0,0.5,0.5
2,steel_input.pd,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
3,abalone_input.pd,0.5,0.5,0.5,0.5,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.0
4,contraceptive_input.pd,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,1.0,...,1.0,1.0,0.5,0.0,1.0,0.5,0.0,0.5,0.0,0.0
5,cars_input.pd,1.0,0.5,0.5,0.5,1.0,1.0,0.5,0.5,0.5,...,0.5,0.5,0.5,1.0,1.0,0.5,0.5,0.5,0.5,1.0
6,students_input.pd,0.5,0.5,0.5,0.5,0.5,0.0,1.0,0.5,1.0,...,0.5,1.0,1.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0
7,adult_input.pd,0.5,0.5,0.5,0.5,0.5,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
8,bank_input.pd,0.5,0.5,0.5,0.5,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.5,0.0,1.0,0.5,0.0,0.5,0.0,0.0
9,hrss_input.pd,0.0,0.0,0.0,0.0,0.5,0.5,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.5


In [37]:
df.mean()

DT-LDA      0.461538
DT-LDAm     0.423077
DT-LR       0.615385
DT-LRm      0.692308
DT-XGB      0.692308
DT-L        0.269231
LDA-LDAm    0.961538
LDA-LR      0.653846
LDA-LRm     0.653846
LDA-XGB     0.538462
LDA-L       0.230769
LDAm-LR     0.653846
LDAm-LRm    0.653846
LDAm-XGB    0.500000
LDAm-L      0.230769
LR-LRm      0.846154
LR-XGB      0.538462
LR-L        0.269231
LRm-XGB     0.576923
LRm-L       0.230769
XGB-L       0.230769
dtype: float64

In [38]:
methods = ["DT","LDA","LDAm","LR","LRm","XGB","L"]

In [39]:
matrix = pd.DataFrame(0.0, index=methods, columns=methods)
matrix

,DT,LDA,LDAm,LR,LRm,XGB,L
DT,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LDA,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LDAm,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LR,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LRm,0.0,0.0,0.0,0.0,0.0,0.0,0.0
XGB,0.0,0.0,0.0,0.0,0.0,0.0,0.0
L,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
matrix["DT"]["LDA"] = 0.461538
matrix["DT"]["LDAm"] = 0.423077
matrix["DT"]["LR"] = 0.615385
matrix["DT"]["LRm"] = 0.692308
matrix["DT"]["XGB"] = 0.692308
matrix["DT"]["L"] = 0.269231
matrix["LDA"]["LDAm"] = 0.961538
matrix["LDA"]["LR"] = 0.653846
matrix["LDA"]["LRm"] = 0.653846
matrix["LDA"]["XGB"] = 0.538462
matrix["LDA"]["L"] = 0.230769
matrix["LDAm"]["LR"] = 0.653846
matrix["LDAm"]["LRm"] = 0.653846
matrix["LDAm"]["XGB"] = 0.500000
matrix["LDAm"]["L"] = 0.230769
matrix["LR"]["LRm"] = 0.846154
matrix["LR"]["XGB"] = 0.538462
matrix["LR"]["L"] = 0.269231
matrix["LRm"]["XGB"] = 0.576923
matrix["LRm"]["L"] = 0.230769
matrix["XGB"]["L"] = 0.230769

In [57]:
matrix = matrix + matrix.T

In [58]:
matrix

,DT,LDA,LDAm,LR,LRm,XGB,L
DT,0.000000,0.461538,0.423077,0.615385,0.692308,0.692308,0.269231
LDA,0.461538,0.000000,0.961538,0.653846,0.653846,0.538462,0.230769
LDAm,0.423077,0.961538,0.000000,0.653846,0.653846,0.500000,0.230769
LR,0.615385,0.653846,0.653846,0.000000,0.846154,0.538462,0.269231
LRm,0.692308,0.653846,0.653846,0.846154,0.000000,0.576923,0.230769
XGB,0.692308,0.538462,0.500000,0.538462,0.576923,0.000000,0.230769
L,0.269231,0.230769,0.230769,0.269231,0.230769,0.230769,0.000000


In [59]:
def printer(n):
    if n == 0.0:
        return " "
    else:
        return "%.2f" % round(n,2)

In [60]:
for index, row in matrix.iterrows():
    row_string = "& "
    for element in row:
        row_string += printer(element)
        row_string += " & "
    row_string = row_string[:-3]
    row_string += "\\\\"
    print(row_string)

&   & 0.46 & 0.42 & 0.62 & 0.69 & 0.69 & 0.27\\
& 0.46 &   & 0.96 & 0.65 & 0.65 & 0.54 & 0.23\\
& 0.42 & 0.96 &   & 0.65 & 0.65 & 0.50 & 0.23\\
& 0.62 & 0.65 & 0.65 &   & 0.85 & 0.54 & 0.27\\
& 0.69 & 0.65 & 0.65 & 0.85 &   & 0.58 & 0.23\\
& 0.69 & 0.54 & 0.50 & 0.54 & 0.58 &   & 0.23\\
& 0.27 & 0.23 & 0.23 & 0.27 & 0.23 & 0.23 &  \\


In [49]:
for index, row in df_c.iterrows():
    print(row.dataset, "&", row.n_features, "&", row.intersection, "&", "%.3f" % round(row.feature_importance_test,3), "&", "%.3f" % round(row.feature_importance_lda_test,3), "&","%.3f" % round(row.PCA_test,3),"&", "%.3f" % round(row.None_test,3), "\\\\")

NameError: name 'df_c' is not defined

In [61]:
df = pd.read_pickle("dataframes/df_intersections_gate_reduction.pd")

In [64]:
df["dataset"] = df["dataset"].apply(lambda x: x.split("_", 1)[0])

In [71]:
df

,dataset,DT-LDA,DT-LDAm,DT-LR,DT-LRm,DT-XGB,DT-L,LDA-LDAm,LDA-LR,LDA-LRm,...,LDAm-LR,LDAm-LRm,LDAm-XGB,LDAm-L,LR-LRm,LR-XGB,LR-L,LRm-XGB,LRm-L,XGB-L
0,breast,0.0,0.0,1.0,1.0,0.5,0.5,1.0,0.0,0.0,...,0.0,0.0,0.5,0.0,1.0,0.5,0.5,0.5,0.5,0.5
1,iris,1.0,1.0,1.0,1.0,1.0,0.5,1.0,1.0,1.0,...,1.0,1.0,1.0,0.5,1.0,1.0,0.5,1.0,0.5,0.5
2,steel,0.0,0.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0
3,abalone,0.5,0.5,0.5,0.5,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.5,0.5,1.0,0.5,0.5,0.5,0.5,0.0
4,contraceptive,1.0,1.0,1.0,1.0,0.5,0.0,1.0,1.0,1.0,...,1.0,1.0,0.5,0.0,1.0,0.5,0.0,0.5,0.0,0.0
5,cars,1.0,0.5,0.5,0.5,1.0,1.0,0.5,0.5,0.5,...,0.5,0.5,0.5,1.0,1.0,0.5,0.5,0.5,0.5,1.0
6,students,0.5,0.5,0.5,0.5,0.5,0.0,1.0,0.5,1.0,...,0.5,1.0,1.0,0.0,0.5,0.5,0.0,1.0,0.0,0.0
7,adult,0.5,0.5,0.5,0.5,0.5,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
8,bank,0.5,0.5,0.5,0.5,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.5,0.0,1.0,0.5,0.0,0.5,0.0,0.0
9,hrss,0.0,0.0,0.0,0.0,0.5,0.5,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.5


In [79]:

for index, row in df.iterrows():
    row_string = ""
    counter = 0
    for element in row:
        counter += 1
        if counter <= 12:
            row_string += str(element)
            row_string += " & "
    row_string = row_string[:-3]
    row_string += "\\\\"
    print(row_string)

breast & 0.0 & 0.0 & 1.0 & 1.0 & 0.5 & 0.5 & 1.0 & 0.0 & 0.0 & 0.5 & 0.0\\
iris & 1.0 & 1.0 & 1.0 & 1.0 & 1.0 & 0.5 & 1.0 & 1.0 & 1.0 & 1.0 & 0.5\\
steel & 0.0 & 0.0 & 0.0 & 0.5 & 0.5 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0\\
abalone & 0.5 & 0.5 & 0.5 & 0.5 & 1.0 & 0.0 & 1.0 & 1.0 & 1.0 & 0.5 & 0.5\\
contraceptive & 1.0 & 1.0 & 1.0 & 1.0 & 0.5 & 0.0 & 1.0 & 1.0 & 1.0 & 0.5 & 0.0\\
cars & 1.0 & 0.5 & 0.5 & 0.5 & 1.0 & 1.0 & 0.5 & 0.5 & 0.5 & 1.0 & 1.0\\
students & 0.5 & 0.5 & 0.5 & 0.5 & 0.5 & 0.0 & 1.0 & 0.5 & 1.0 & 1.0 & 0.0\\
adult & 0.5 & 0.5 & 0.5 & 0.5 & 0.5 & 0.0 & 1.0 & 1.0 & 1.0 & 1.0 & 0.0\\
bank & 0.5 & 0.5 & 0.5 & 0.5 & 1.0 & 0.0 & 1.0 & 1.0 & 1.0 & 0.5 & 0.0\\
hrss & 0.0 & 0.0 & 0.0 & 0.0 & 0.5 & 0.5 & 1.0 & 1.0 & 1.0 & 0.0 & 0.0\\
occupancy & 0.0 & 0.0 & 1.0 & 1.0 & 0.5 & 0.5 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0\\
pdm6 & 0.5 & 0.5 & 0.5 & 1.0 & 0.5 & 0.5 & 1.0 & 1.0 & 0.5 & 0.5 & 0.5\\
banknote & 0.5 & 0.5 & 1.0 & 1.0 & 1.0 & 0.0 & 1.0 & 0.5 & 0.5 & 0.5 & 0.5\\


In [80]:
for index, row in df.iterrows():
    row_string = ""
    counter = 0
    for element in row:
        counter += 1
        if counter == 1 or counter >12:
            row_string += str(element)
            row_string += " & "
    row_string = row_string[:-3]
    row_string += "\\\\"
    print(row_string)

breast & 0.0 & 0.0 & 0.5 & 0.0 & 1.0 & 0.5 & 0.5 & 0.5 & 0.5 & 0.5\\
iris & 1.0 & 1.0 & 1.0 & 0.5 & 1.0 & 1.0 & 0.5 & 1.0 & 0.5 & 0.5\\
steel & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.5 & 0.0 & 0.0 & 0.0\\
abalone & 1.0 & 1.0 & 0.5 & 0.5 & 1.0 & 0.5 & 0.5 & 0.5 & 0.5 & 0.0\\
contraceptive & 1.0 & 1.0 & 0.5 & 0.0 & 1.0 & 0.5 & 0.0 & 0.5 & 0.0 & 0.0\\
cars & 0.5 & 0.5 & 0.5 & 1.0 & 1.0 & 0.5 & 0.5 & 0.5 & 0.5 & 1.0\\
students & 0.5 & 1.0 & 1.0 & 0.0 & 0.5 & 0.5 & 0.0 & 1.0 & 0.0 & 0.0\\
adult & 1.0 & 1.0 & 1.0 & 0.0 & 1.0 & 1.0 & 0.0 & 1.0 & 0.0 & 0.0\\
bank & 1.0 & 1.0 & 0.5 & 0.0 & 1.0 & 0.5 & 0.0 & 0.5 & 0.0 & 0.0\\
hrss & 1.0 & 1.0 & 0.0 & 0.0 & 1.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.5\\
occupancy & 0.0 & 0.0 & 0.0 & 0.0 & 1.0 & 0.5 & 0.5 & 0.5 & 0.5 & 0.5\\
pdm6 & 1.0 & 0.5 & 0.5 & 0.5 & 0.5 & 0.5 & 0.5 & 0.5 & 0.5 & 0.0\\
banknote & 0.5 & 0.5 & 0.5 & 0.5 & 1.0 & 1.0 & 0.0 & 1.0 & 0.0 & 0.0\\


In [84]:
str1 = ""
for x in df.columns:
    str1+= x
    str1 += " & "
print(str1)

dataset & DT-LDA & DT-LDAm & DT-LR & DT-LRm & DT-XGB & DT-L & LDA-LDAm & LDA-LR & LDA-LRm & LDA-XGB & LDA-L & LDAm-LR & LDAm-LRm & LDAm-XGB & LDAm-L & LR-LRm & LR-XGB & LR-L & LRm-XGB & LRm-L & XGB-L & 
